In [ ]:
!git clone https://github.com/cg123/mergekit.git
%cd mergekit
%pip install -e .

In [ ]:
OUTPUT_PATH = "./merged"  # folder to store the result in
LORA_MERGE_CACHE = "/tmp"  # change if you want to keep these for some reason
CONFIG_YML = "./examples/gradient-slerp.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [ ]:
# actually do merge
import torch
import mergekit.scripts.run_yaml

mergekit.scripts.run_yaml.main(
    CONFIG_YML,
    OUTPUT_PATH,
    LORA_MERGE_CACHE,
    cuda=torch.cuda.is_available(),
    copy_tokenizer=COPY_TOKENIZER,
    lazy_unpickle=LAZY_UNPICKLE,
    low_cpu_memory=LOW_CPU_MEMORY,
)